In [ ]:
!pip install -U FlagEmbedding peft faiss-cpu LM_Cocktail

  Using cached FlagEmbedding-1.3.4.tar.gz (163 kB)
  Preparing metadata (setup.py) ... done
  Using cached peft-0.15.1-py3-none-any.whl.metadata (13 kB)
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.0/411.0 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   

In [ ]:
!pip install deepspeed
!pip install flash-attn --no-build-isolation

In [ ]:
!git clone https://github.com/FlagOpen/FlagEmbedding.git
# !pip install -U FlagEmbedding[finetune]

Cloning into 'FlagEmbedding'...
remote: Enumerating objects: 10816, done.
remote: Counting objects: 100% (280/280), done.
remote: Compressing objects: 100% (132/132), done.
remote: Total 10816 (delta 177), reused 211 (delta 148), pack-reused 10536 (from 2)
Receiving objects: 100% (10816/10816), 38.90 MiB | 23.13 MiB/s, done.
Resolving deltas: 100% (5967/5967), done.


우선 학습을 위해 임의의 데이터셋을 준비합니다. 데이터셋의 형식은 반드시 jsonl 형태여야만 합니다. 각 데이터는 "query", "pos", "neg"로 구성되며, 이때 "query"는 검색어, "pos"는 해당 검색어와 연관된 문서들이 포함된 리스트, "neg"는 해당 검색어와 연관되지 않은 문서들이 포함된 리스트입니다.

여기서는 저자가 임의로 데이터를 준비했지만, 실제 파인 튜닝 상황에서는 사용자는 실제로 이런 검색어가 입력되었을 때(query), 이런 문서들이 검색되었으면 하며(pos), 이런 문서들은 검색되지 않았으면 한다(neg)에 해당하는 데이터들을 직접 구축해야만 합니다.

In [ ]:
import json

# 번역된 데이터
translated_data = [
    {"query": "다섯 명의 여성이 해변을 따라 플립플롭을 신고 걸어간다.", "pos": ["플립플롭을 신은 몇몇 여성들이 해변을 따라 걸어가고 있다"], "neg": ["4명의 여성이 해변에 앉아 있다.", "1996년에 개혁이 있었다.", "그녀는 자신의 기록을 정정하기 위해 법정에 가지 않을 것이다.", "그 남자는 하와이에 대해 이야기하고 있다.", "한 여성이 밖에 서 있다.", "전투는 끝났다.", "한 무리의 사람들이 배구를 하고 있다."]},
    {"query": "한 여성이 높은 절벽 위에서 한 발로 서서 강을 내려다보고 있다.", "pos": ["한 여성이 절벽 위에 서 있다."], "neg": ["한 여성이 의자에 앉아 있다.", "조지 부시는 공화당원들에게 최고 고문들의 조언에 반하여 이 어리석은 생각을 고려조차 하지 않겠다고 말했다.", "그 가족은 무너지고 있었다.", "아무도 회의에 나타나지 않았다", "한 소년이 밖에서 모래를 가지고 놀고 있다.", "전보를 받자마자 끝났다.", "한 아이가 자기 방에서 책을 읽고 있다."]},
    {"query": "두 여성이 악기를 연주하고 있다; 한 명은 클라리넷, 다른 한 명은 바이올린을 연주한다.", "pos": ["몇 사람이 곡을 연주하고 있다."], "neg": ["두 여성이 기타와 드럼을 연주하고 있다.", "한 남자가 산을 스키를 타고 내려가고 있다.", "살인자가 생각했던 때에 치명적인 용량이 투여되지 않았다.", "자전거를 타고 있는 사람", "그 소녀는 아치길에 기대어 서 있다.", "한 무리의 여성들이 소파 오페라를 보고 있다.", "사람들은 나이가 들어도 절대 잊지 않는다."]},
    {"query": "파란색 탱크톱을 입은 소녀가 앉아서 세 마리의 개를 지켜보고 있다.", "pos": ["한 소녀가 파란색을 입고 있다."], "neg": ["한 소녀가 세 마리의 고양이와 함께 있다.", "사람들이 장례 행렬을 지켜보고 있다.", "그 아이는 검은색을 입고 있다.", "공립학교에서 우리에게 재정은 문제이다.", "수영장에 있는 아이들.", "폭행당하는 것은 진정시키는 일이다.", "나는 18살에 심각한 문제에 직면했다."]},
    {"query": "노란 개가 숲길을 따라 달리고 있다.", "pos": ["개가 달리고 있다"], "neg": ["고양이가 달리고 있다", "스틸은 그녀의 원래 이야기를 지키지 않았다.", "이 규칙은 사람들이 자녀 양육비를 내는 것을 막는다.", "조끼를 입은 남자가 차 안에 앉아 있다.", "검은 옷을 입고 흰색 반다나와 선글라스를 낀 사람이 버스 정류장에서 기다리고 있다.", "글로브나 메일 중 어느 쪽도 캐나다의 현재 도로 체계 상태에 대해 언급하지 않았다.", "스프링 크릭 시설은 오래되고 구식이다."]},
    {"query": "각 단계에서의 필수 활동과 그 활동들과 관련된 중요한 요소들을 설명한다.", "pos": ["필수 활동에 대한 중요 요소들이 설명되어 있다."], "neg": ["중요한 활동들을 설명하지만 그 활동들과 관련된 중요한 요소들에 대한 규정은 없다.", "사람들이 항의하기 위해 모여 있다.", "주 정부는 당신이 그렇게 하기를 선호할 것이다.", "한 소녀가 한 소년 옆에 앉아 있다.", "두 남성이 공연하고 있다.", "아무도 뛰고 있지 않다", "콘라드는 머리를 맞도록 음모를 꾸미고 있었다."]},
    {"query": "한 남자가 레스토랑에서 연설을 하고 있다.", "pos": ["한 사람이 연설을 하고 있다."], "neg": ["그 남자는 테이블에 앉아 음식을 먹고 있다.", "이것은 확실히 승인이 아니다.", "그들은 은퇴 때문에 집을 팔았지, 대출 때문이 아니다.", "미주리 주의 인장은 완벽하다.", "누군가가 손을 들고 있다.", "한 운동선수가 1500미터 수영 경기에 참가하고 있다.", "두 남자가 마술 쇼를 보고 있다."]},
    {"query": "인디언들이 코트를 입고 음식과 음료를 가지고 모임을 갖고 있다.", "pos": ["인디언 그룹이 음식과 음료를 가지고 모임을 갖고 있다"], "neg": ["인디언 그룹이 장례식을 하고 있다", "이것은 팔마의 큰 투우장에서 겨울 오후에만 공연된다.", "올바른 정보는 법률 서비스 관행과 사법 체계를 강화할 수 있다.", "한편, 본토는 인구가 없었다.", "두 아이가 자고 있다.", "어부가 원숭이를 잡으려고 하고 있다", "사람들이 기차 안에 있다"]},
    {"query": "보라색 머리를 한 여성이 밖에서 자전거를 타고 있다.", "pos": ["한 여성이 자전거를 타고 있다."], "neg": ["한 여성이 공원에서 조깅을 하고 있다.", "그 거리는 하얀색으로 칠해진 집들로 가득했다.", "한 그룹이 안에서 영화를 보고 있다.", "소풍에서 남자들이 스테이크를 자르고 있다", "여러 명의 요리사들이 앉아서 음식에 대해 이야기하고 있다.", "위원회는 중요한 대안들이 고려되지 않았다고 지적한다.", "우리는 장작이 다 떨어져서 불을 위해 소나무 바늘을 사용해야 했다."]},
    {"query": "한 남자가 도시 거리에서 인력거로 두 여성을 끌고 있다.", "pos": ["한 남자가 도시에 있다."], "neg": ["한 남자가 비행기 조종사이다.", "그것은 지루하고 평범하다.", "아침 햇살이 밝게 비치고 따뜻했다.", "두 사람이 부두에서 뛰어내렸다.", "사람들이 우주선 발사를 보고 있다.", "테레사 수녀는 쉬운 선택이다.", "원하는 속도로 갈 수 있는 것은 가치가 있다."]}
]

# JSONL 파일로 저장
with open('toy_finetune_data.jsonl', 'w', encoding='utf-8') as f:
    for item in translated_data:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')

print("데이터가 'toy_finetune_data.jsonl' 파일로 저장되었습니다.")


데이터가 'toy_finetune_data.jsonl' 파일로 저장되었습니다.


데이터를 자동으로 최적화하는 과정을 거쳐봅시다.

!python -m FlagEmbedding.baai_general_embedding.finetune.hn_mine \
--model_name_or_path BAAI/bge-m3 \
--input_file toy_finetune_data.jsonl \
--output_file toy_finetune_data_minedHN.jsonl \
--range_for_sampling 2-200 \
--negative_number 15 \
--use_gpu_for_searching


input_file: 최적화 하기를 원하는 여러분들이 파인튜닝을 위해 준비한 JSON 데이터입니다. 위 코드를 실행하면 각 query에 대해서 실제로 bge-m3로 유사도가 높은 상위 k개의 문서를 검색하고, 이 상위 k개 문서에서 무작위로 네거티브 샘플링을 수행합니다 (pos 문서는 제외).

output_file: 네거티브 샘플링을 수행하여 여러분들의 데이터를 학습하기에 최적화 시킨 후의 JSON 데이터를 저장할 경로입니다.

negative_number: 샘플링할 네거티브의 수입니다.

range_for_sampling: 네거티브를 샘플링할 범위입니다. 예를 들어, 2-100은 상위 2위부터 200위 문서 중에서 negative_number만큼의 네거티브를 샘플링한다는 의미입니다. 네거티브의 난이도를 낮추기 위해 더 큰 값을 설정할 수 있습니다 (예: 상위 60-300위 문서에서 네거티브를 샘플링하려면 60-300으로 설정)

use_gpu_for_searching: 네거티브 검색에 faiss-gpu를 사용할지 여부입니다.

In [ ]:
!python /content/FlagEmbedding/research/baai_general_embedding/finetune/hn_mine.py \
--model_name_or_path BAAI/bge-m3 \
--input_file toy_finetune_data.jsonl \
--output_file toy_finetune_data_minedHN.jsonl \
--range_for_sampling 2-200 \
--negative_number 15 \
# --use_gpu_for_searching


2025-04-15 02:01:36.653121: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744682496.674244    3650 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744682496.681561    3650 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-15 02:01:36.703717: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[2025-04-15 02:01:41,564] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detec

In [ ]:
import json

data = []
with open('toy_finetune_data.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        item = json.loads(line)
        data.append(item)

data[0]


{'query': '다섯 명의 여성이 해변을 따라 플립플롭을 신고 걸어간다.',
 'pos': ['플립플롭을 신은 몇몇 여성들이 해변을 따라 걸어가고 있다'],
 'neg': ['4명의 여성이 해변에 앉아 있다.',
  '1996년에 개혁이 있었다.',
  '그녀는 자신의 기록을 정정하기 위해 법정에 가지 않을 것이다.',
  '그 남자는 하와이에 대해 이야기하고 있다.',
  '한 여성이 밖에 서 있다.',
  '전투는 끝났다.',
  '한 무리의 사람들이 배구를 하고 있다.']}

In [ ]:
import json

data = []
with open('toy_finetune_data_minedHN.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        item = json.loads(line)
        data.append(item)

data[0]

{'query': '다섯 명의 여성이 해변을 따라 플립플롭을 신고 걸어간다.',
 'pos': ['플립플롭을 신은 몇몇 여성들이 해변을 따라 걸어가고 있다'],
 'neg': ['한 아이가 자기 방에서 책을 읽고 있다.',
  '원하는 속도로 갈 수 있는 것은 가치가 있다.',
  '사람들이 기차 안에 있다',
  '살인자가 생각했던 때에 치명적인 용량이 투여되지 않았다.',
  '이것은 확실히 승인이 아니다.',
  '이것은 팔마의 큰 투우장에서 겨울 오후에만 공연된다.',
  '두 여성이 기타와 드럼을 연주하고 있다.',
  '한 소년이 밖에서 모래를 가지고 놀고 있다.',
  '한 그룹이 안에서 영화를 보고 있다.',
  '두 아이가 자고 있다.',
  '위원회는 중요한 대안들이 고려되지 않았다고 지적한다.',
  '그녀는 자신의 기록을 정정하기 위해 법정에 가지 않을 것이다.',
  '콘라드는 머리를 맞도록 음모를 꾸미고 있었다.',
  '검은 옷을 입고 흰색 반다나와 선글라스를 낀 사람이 버스 정류장에서 기다리고 있다.',
  '한 여성이 자전거를 타고 있다.']}

per_device_train_batch_size: 학습 시 배치 크기입니다. 대부분의 경우, 더 큰 배치 크기가 더 강력한 성능을 가져옵니다. GPU 성능에 달려있으며, 여기서는 토이 프로젝트이고 데이터가 소량이므로 1을 선택했습니다.

train_group_size: 학습 시 쿼리당 긍정 및 부정 예제의 수입니다. 항상 하나의 긍정 예제가 있으므로, 이 인자는 부정 예제의 수를 제어합니다 (부정 예제 수 = train_group_size - 1). 부정 예제의 수가 데이터의 "neg":List[str]에 있는 부정 예제 수보다 크지 않아야 함에 주의하세요. 이 그룹의 부정 예제 외에도, 배치 내 부정 예제도 파인튜닝에 사용됩니다.

negatives_cross_device: 모든 GPU에서 부정 예제를 공유합니다. 이 인자는 부정 예제의 수를 확장합니다.

learning_rate: 모델에 적합한 값을 선택하세요. 대규모/기본/소규모 모델에 대해 각각 1e-5/2e-5/3e-5를 추천합니다.

temperature: 유사도 점수의 분포에 영향을 미칩니다. 권장 값: 0.01-0.1.

query_max_len: 쿼리의 최대 길이입니다. 데이터의 평균 쿼리 길이에 따라 설정해 주세요.

passage_max_len: 문장의 최대 길이입니다. 데이터의 평균 문장 길이에 따라 설정해 주세요.

query_instruction_for_retrieval: 쿼리에 대한 지시사항으로, 각 쿼리에 추가됩니다. 아무것도 추가하지 않으려면 ""로 설정할 수 있습니다.

use_inbatch_neg: 같은 배치 내의 문장들을 부정 예제로 사용합니다. 기본값은 True입니다.

save_steps: 체크포인트를 저장할 학습 단계 간격을 설정합니다.
학습이 끝나면 checkpoint라는 디렉토리에 학습 모델이 저장됩니다.

In [ ]:
!torchrun --nproc_per_node 1 \
    -m FlagEmbedding.finetune.embedder.encoder_only.base \
    --model_name_or_path BAAI/bge-large-en-v1.5 \
    --cache_dir ./cache/model \
    --train_data /content/toy_finetune_data.jsonl \
    --cache_path ./cache/data \
    --train_group_size 1 \
    --query_max_len 512 \
    --passage_max_len 512 \
    --pad_to_multiple_of 8 \
    --query_instruction_for_retrieval 'Represent this sentence for searching relevant passages: ' \
    --query_instruction_format '{}{}' \
    --knowledge_distillation False \
    --output_dir ./test_encoder_only_base_bge-m3 \
    --overwrite_output_dir \
    --learning_rate 1e-5 \
    --fp16 \
    --num_train_epochs 1 \
    --per_device_train_batch_size 1 \
    --dataloader_drop_last True \
    --warmup_ratio 0.1 \
    --gradient_checkpointing \
    --logging_steps 1 \
    --save_steps 1000 \
    --negatives_cross_device \
    --temperature 0.02 \
    --sentence_pooling_method cls \
    --normalize_embeddings True \
    --kd_loss_type kl_div \
    --deepspeed /content/FlagEmbedding/FlagEmbedding/finetune/ds_stage0.json

2025-04-15 02:44:13.345712: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744685053.367708   15829 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744685053.374219   15829 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[2025-04-15 02:44:16,955] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2025-04-15 02:44:18,121] [INFO] [comm.py:658:init_distributed] cdb=None
[2025-04-15 02:44:18,121] [INFO] [comm.py:689:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
04/15/2025 02:44:18 - WARNING - FlagEmbedding.abc.finetune.embedder.AbsRunner -   Process rank: 0, device: cuda:0, n_gpu: 1

기본 BGE 모델을 파인튜닝하면 목표 작업에서의 성능을 향상시킬 수 있지만, 학습한 유형의 도메인을 넘어선 모델의 일반적인 능력이 저하될 수도 있습니다. 예를 들어서 금융 분야의 도메인의 데이터를 학습시키면, 금융 분야에서는 성능이 오를지 몰라도 그 외의 도메인의 데이터에서는 성능 저하가 심각하게 발생하는 경우 등이 있습니다. 이 경우, 파인 튜닝 모델과 기존 모델을 병합하여 일반화 성능을 어느 정도 유지하면서도 파인 튜닝 모델 성능을 가져올 수 있습니다.

두 모델 중 어떤 모델의 능력을 더 우선시 하면서 융합할 것인지를 정할 수 있는데, 이는 아래의 코드에서 weights의 값을 조절하여 결정합니다. 예컨대 [0.5, 0.5]는 두 모델의 비중을 50:50으로 가져가는 것을 의미합니다.

In [ ]:
from LM_Cocktail import mix_models, mix_models_with_data

# 기존 모델과 파인 튜닝 모달을 융합하여 mixed_model_1 디렉토리에 융합 모델을 저장.
model = mix_models(
    model_names_or_paths=["BAAI/bge-m3", "/content/checkpoint"],
    model_type='encoder',
    weights=[0.5, 0.5],  # 가중치를 조절하세요.
    output_path='./mixed_model_1')